# Spotify bővített streaming előzmények – részletes elemzés (magyar kommentekkel)

Ebben a szekcióban a Spotify által letölthető „Extended Streaming History” JSON fájlokat elemezzük. 
A cellákban magyar nyelvű, részletes kommentek magyarázzák, hogy mit, miért és hogyan csinálunk.


A következő cellák először betöltik és egységesítik az adatokat, majd elkészítik a fenti lekérdezéseket.

In [57]:
# Könyvtárak importálása
# Itt hozzuk be a szükséges Python könyvtárakat.
# - json: a JSON állományok betöltéséhez
# - pandas (pd): táblázatos adatkezeléshez
# - numpy (np): numerikus segédfüggvényekhez
# - pathlib Path: platformfüggetlen útvonalkezeléshez
# - datetime: dátum-idő segédműveletekhez
# - re: reguláris kifejezések (például platform string tisztításához)
import json
from pathlib import Path
import re
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
from IPython.display import display

# Kimeneti beállítások átláthatóságért
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

In [ ]:
# Adatfájlok helye – feltételezzük, hogy a notebook mappájában van a Spotify_Extended_Streaming_History mappa.
# Ha máshol lenne, itt módosítsd az elérési utat.
DATA_DIR = Path.cwd() / "Spotify_Extended_Streaming_History"

# Mely JSON-okat vegyük figyelembe: Audio és opcionálisan Video.
# A kiterjesztett előzményekben a podcastok is az Audio fájlokban vannak soronként elkülönítve
# (episode_* mezők), ezért elsősorban az "Audio" fájlokat használjuk. A Video fájl opcionális.
AUDIO_PAT = "Streaming_History_Audio_*.json"
VIDEO_PAT = "Streaming_History_Video_*.json"

# Segédfüggvény: biztonságos JSON beolvasás listaként.
# A Spotify JSON export tipikusan egy listát tartalmaz, ahol minden elem egy lejátszási sor.
def read_json_list(path: Path) -> list:
    with path.open("r", encoding="utf-8") as f:
        return json.load(f)

# Az összes releváns fájl összegyűjtése és egyetlen DataFrame-be fűzése.
rows = []
for p in sorted(DATA_DIR.glob(AUDIO_PAT)):
    try:
        rows.extend(read_json_list(p))
    except Exception as e:
        print(f"Hiba a fájl olvasásakor: {p.name} -> {e}")

# Video fájl opcionális, ha jelen van, csatoljuk.
for p in sorted(DATA_DIR.glob(VIDEO_PAT)):
    try:
        rows.extend(read_json_list(p))
    except Exception as e:
        print(f"Hiba a fájl olvasásakor (VIDEO): {p.name} -> {e}")

# Nyers DataFrame létrehozása
raw = pd.DataFrame(rows)
print(f"Sorok száma összesen: {len(raw):,}")
raw.head(3)

Sorok száma összesen: 134,514


,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2018-07-30T15:39:56Z,"Android OS 5.1 API 22 (LENOVO, Lenovo A7010a48)",11648,HU,176.63.30.118,None,None,None,None,#0 - Pilot,The Misfits Podcast,spotify:episode:67vcPiq854ZJhZUjEobsL0,None,None,None,None,clickrow,unexpected-exit-while-paused,False,False,False,NaN,False
1,2018-07-31T11:57:58Z,"Android OS 5.1 API 22 (LENOVO, Lenovo A7010a48)",50665,HU,176.63.30.166,None,None,None,None,#0 - Pilot,The Misfits Podcast,spotify:episode:67vcPiq854ZJhZUjEobsL0,None,None,None,None,appload,endplay,False,False,False,NaN,False
2,2018-07-31T13:18:15Z,"Android OS 5.1 API 22 (LENOVO, Lenovo A7010a48)",1133551,HU,37.76.106.188,None,None,None,None,#1 - The Poland Story,The Misfits Podcast,spotify:episode:0FqoVwr8OOHVRTUkm2Q5mL,None,None,None,None,clickrow,unexpected-exit-while-paused,False,False,False,NaN,False


In [59]:
# Oszlopok egységesítése és kényelmi mezők képzése
# - ts: ISO8601 időbélyeg -> pandas datetime (UTC). Később helyi időzónát is választhatsz, ha szeretnéd.
# - ms_played: milliszekundum -> másodperc, perc (könnyebb aggregálni)
# - content_type: {track|podcast|video|audiobook|unknown} – a mezőkből következtetve
# - track/episode azonosítók és meta, ha elérhető (név, előadó, album)
# - dátum-származtatott mezők: date, year, month, dow (0=hétfő), hour

df = raw.copy()

# Idő
df["ts"] = pd.to_datetime(df["ts"], utc=True, errors="coerce")

# Idő alapú kényelmi mezők
df["date"] = df["ts"].dt.date
df["year"] = df["ts"].dt.year
df["month"] = df["ts"].dt.to_period("M").astype(str)  # pl. "2023-07"
df["dow"] = df["ts"].dt.dayofweek  # 0=hétfő, 6=vasárnap
df["hour"] = df["ts"].dt.hour

# Időtartamok konverziója
df["sec"] = (df["ms_played"].fillna(0) / 1000).astype(float)
df["min"] = df["sec"] / 60.0

# Tartalom típus becslése a mezők alapján
# Heuriszta: ha van episode_name vagy spotify_episode_uri -> podcast
# ha van master_metadata_track_name vagy spotify_track_uri -> track
# ha kifejezetten Video fájlból jött, jelöljük video-ként (itt a mezők sorrendje alapján bírálunk)
# ha audiobook mezők nem üresek -> audiobook

def infer_type(row):
    if pd.notna(row.get("audiobook_title")) or pd.notna(row.get("audiobook_chapter_uri")):
        return "audiobook"
    if pd.notna(row.get("episode_name")) or pd.notna(row.get("spotify_episode_uri")):
        return "podcast"
    if pd.notna(row.get("master_metadata_track_name")) or pd.notna(row.get("spotify_track_uri")):
        return "track"
    # fallback: ha a Video fájlon belül lennénk, jöhetne jelölés, de a forrásfájlt nem őriztük meg soronként
    return "unknown"

df["content_type"] = df.apply(infer_type, axis=1)

# Zeneszám meta
df["track"] = df.get("master_metadata_track_name")
df["artist"] = df.get("master_metadata_album_artist_name")
df["album"] = df.get("master_metadata_album_album_name")

# Podcast meta
df["podcast_episode"] = df.get("episode_name")
df["podcast_show"] = df.get("episode_show_name")

# Platform egyszerűsítése: OS / eszköz család kinyerése
# Példák: "Android OS 5.1 API 22 (LENOVO, Lenovo A7010a48)" -> "Android"
#         "iOS 16.5 (Apple, iPhone14,7)" -> "iOS"
#         "Windows 10 (PC)" -> "Windows"
#         "Mac OS X 14.4.1 (x86_64)" -> "Mac"

def simplify_platform(s: str) -> str:
    if not isinstance(s, str):
        return "Other"
    s_low = s.lower()
    if "android" in s_low:
        return "Android"
    if "ios" in s_low or "iphone" in s_low or "ipad" in s_low:
        return "iOS"
    if "windows" in s_low:
        return "Windows"
    if "mac os" in s_low or "macos" in s_low or "os x" in s_low:
        return "Mac"
    if "linux" in s_low:
        return "Linux"
    return "Other"

df["platform_simple"] = df["platform"].apply(simplify_platform)

# Hétvége jelölés a hőtérképhez
df["is_weekend"] = df["dow"].isin([5, 6])  # 5=szombat, 6=vasárnap

# Negatív/hibás idők kiszűrése (néha ms_played lehet 0 vagy nagyon kicsi)
df = df[df["sec"] >= 0]

# Hiányos időbélyegek eldobása
df = df.dropna(subset=["ts"]) 

print(df.shape)
df.head(5)

c:\Users\gondo\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


(134514, 38)


,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode,date,year,month,dow,hour,sec,min,content_type,track,artist,album,podcast_episode,podcast_show,platform_simple,is_weekend
0,2018-07-30 15:39:56+00:00,"Android OS 5.1 API 22 (LENOVO, Lenovo A7010a48)",11648,HU,176.63.30.118,None,None,None,None,#0 - Pilot,The Misfits Podcast,spotify:episode:67vcPiq854ZJhZUjEobsL0,None,None,None,None,clickrow,unexpected-exit-while-paused,False,False,False,NaN,False,2018-07-30,2018,2018-07,0,15,11.648,0.194133,podcast,None,None,None,#0 - Pilot,The Misfits Podcast,Android,False
1,2018-07-31 11:57:58+00:00,"Android OS 5.1 API 22 (LENOVO, Lenovo A7010a48)",50665,HU,176.63.30.166,None,None,None,None,#0 - Pilot,The Misfits Podcast,spotify:episode:67vcPiq854ZJhZUjEobsL0,None,None,None,None,appload,endplay,False,False,False,NaN,False,2018-07-31,2018,2018-07,1,11,50.665,0.844417,podcast,None,None,None,#0 - Pilot,The Misfits Podcast,Android,False
2,2018-07-31 13:18:15+00:00,"Android OS 5.1 API 22 (LENOVO, Lenovo A7010a48)",1133551,HU,37.76.106.188,None,None,None,None,#1 - The Poland Story,The Misfits Podcast,spotify:episode:0FqoVwr8OOHVRTUkm2Q5mL,None,None,None,None,clickrow,unexpected-exit-while-paused,False,False,False,NaN,False,2018-07-31,2018,2018-07,1,13,1133.551,18.892517,podcast,None,None,None,#1 - The Poland Story,The Misfits Podcast,Android,False
3,2018-07-31 15:39:00+00:00,"Android OS 5.1 API 22 (LENOVO, Lenovo A7010a48)",1133551,HU,178.48.68.57,None,None,None,None,#1 - The Poland Story,The Misfits Podcast,spotify:episode:0FqoVwr8OOHVRTUkm2Q5mL,None,None,None,None,clickrow,unexpected-exit-while-paused,False,False,False,NaN,False,2018-07-31,2018,2018-07,1,15,1133.551,18.892517,podcast,None,None,None,#1 - The Poland Story,The Misfits Podcast,Android,False
4,2018-08-01 10:33:50+00:00,"Android OS 5.1 API 22 (LENOVO, Lenovo A7010a48)",179895,HU,176.63.30.166,None,None,None,None,#1 - The Poland Story,The Misfits Podcast,spotify:episode:0FqoVwr8OOHVRTUkm2Q5mL,None,None,None,None,appload,unexpected-exit-while-paused,False,False,False,NaN,False,2018-08-01,2018,2018-08,2,10,179.895,2.998250,podcast,None,None,None,#1 - The Poland Story,The Misfits Podcast,Android,False


In [60]:
# 10 legnépszerűbb előadó 2025-ben – bővített statisztikákkal

df_2025 = df[df["year"] == 2025].copy()

# Csoportosítás előadónként
artists_2025 = (
    df_2025.groupby("master_metadata_album_artist_name")
           .agg(
               Lejátszások_száma=("master_metadata_album_artist_name", "count"),
               Összes_perc=("min", "sum"),
               Napok_száma=("date", "nunique")
           )
           .reset_index()
           .rename(columns={"master_metadata_album_artist_name": "Előadó"})
)

# Átlag perc/nap
artists_2025["Átlag_perc_nap"] = artists_2025["Összes_perc"] / artists_2025["Napok_száma"].replace(0, np.nan)

# Részesedés %
total_minutes_2025 = artists_2025["Összes_perc"].sum()
artists_2025["Részesedés_%"] = 100 * artists_2025["Összes_perc"] / total_minutes_2025

# Top 10 előadó
top10_artists_2025 = (
    artists_2025.sort_values("Összes_perc", ascending=False)
                .head(10)
                .round({"Összes_perc": 1, "Átlag_perc_nap": 1, "Részesedés_%": 2})
)

display(top10_artists_2025)


,Előadó,Lejátszások_száma,Összes_perc,Napok_száma,Átlag_perc_nap,Részesedés_%
229,Eminem,3348,7010.3,253,27.7,13.75
505,NF,2095,3662.9,251,14.6,7.19
605,Ryan Caraveo,1554,2618.4,242,10.8,5.14
354,Joyner Lucas,904,1757.1,209,8.4,3.45
816,mgk,1276,1662.7,233,7.1,3.26
356,Juice WRLD,927,1496.3,217,6.9,2.94
761,Witt Lowry,615,1071.4,207,5.2,2.10
561,Post Malone,629,1051.4,203,5.2,2.06
370,Kanye West,638,1002.0,204,4.9,1.97
375,Kendrick Lamar,649,978.7,203,4.8,1.92


In [61]:
# 1) Legtöbbet hallgatott előadók (csak zeneszámok)
# Cél: Megnézni, kik azok az előadók, akiket a legtöbbet hallgattunk.
# Mutatók: összes perc, egyedi napok száma, átlag perc/nap, részesedés az összes zenei időből.

tracks = df[df["content_type"] == "track"].copy()

# Előadónként csoportosítás és mutatók számítása
a1 = (
    tracks.groupby("artist")
          .agg(
              total_min=("min", "sum"),       # összes perc az adott előadótól
              days=("date", "nunique"),       # hány egyedi napon hallgattuk
              plays=("track", "count")        # lejátszások száma (összes sor)
          )
          .reset_index()
)

# Csak olyan előadókat tartunk meg, ahol az "artist" érték nem hiányzik és nem üres
a1 = a1[a1["artist"].notna() & (a1["artist"] != "")]

# Összes hallgatott perc (minden előadóra)
total_track_min = a1["total_min"].sum()

# Új mutatók
a1["avg_min_per_day"] = a1["total_min"] / a1["days"].replace(0, np.nan)  # átlag perc/nap
a1["share_%"] = 100 * a1["total_min"].div(total_track_min).fillna(0)     # részesedés az összesből %

# Oszlopok magyar elnevezése a kimenethez
a1 = a1.rename(columns={
    "artist": "Előadó",
    "total_min": "Összes perc",
    "days": "Hallgatott különböző napok száma",
    "plays": "Lejátszások száma",
    "avg_min_per_day": "Átlag perc/nap",
    "share_%": "Részesedés (%)"
})

# Top 20 előadó – az összes perc és a hallgatott napok alapján sorba rendezve
res1 = a1.sort_values(["Összes perc", "Hallgatott különböző napok száma"], ascending=False).head(30)

# Szebb, kerekített megjelenítés
res1.round({
    "Összes perc": 1,
    "Átlag perc/nap": 1,
    "Részesedés (%)": 2
})


,Előadó,Összes perc,Hallgatott különböző napok száma,Lejátszások száma,Átlag perc/nap,Részesedés (%)
719,Eminem,22989.2,1143,13012,20.1,10.55
1554,NF,18776.3,1150,11305,16.3,8.62
1878,Ryan Caraveo,8406.3,634,4489,13.3,3.86
2493,mgk,7741.6,824,4613,9.4,3.55
1145,Kanye West,5908.0,802,3178,7.4,2.71
1164,Kendrick Lamar,5584.4,804,2800,6.9,2.56
1103,Joyner Lucas,5318.7,767,2758,6.9,2.44
2338,Witt Lowry,4330.7,547,1881,7.9,1.99
1226,Krúbi,3917.0,626,2037,6.3,1.80
306,Beton.Hofi,3866.8,617,2215,6.3,1.77


In [62]:
# 2) Átugrási ráta zenéknél – előadónként
# Cél: Megnézni, hogy mely előadók számait ugorják át leggyakrabban, illetve legkevésbé.
# A "skipped" mező jelzi, hogy átugrották-e a dalt (True/False).
# Csak azokat az előadókat nézzük, akiknél elég adat van (min. 50 sor).

sk = tracks.copy()

# Egyes exportokból hiányozhat a "skipped" oszlop.
# Ha hiányzik vagy NaN van benne, alapértelmezésként False-t veszünk (nem ugrották át).
sk["skipped"] = sk.get("skipped", False).fillna(False).astype(bool)

# Előadónként csoportosítunk, és kiszámoljuk a főbb mutatókat.
sk_stats = (
    sk.groupby("artist")
      .agg(
          n=("skipped", "size"),        # összes szám (mintaszám)
          skips=("skipped", "sum"),     # átugrott számok darabszáma
          total_min=("min", "sum")      # teljes hossz percben
      )
      .reset_index()
)

# Csak azokat az előadókat tartjuk meg, ahol legalább 150 szám szerepel.
sk_stats = sk_stats[sk_stats["n"] >= 150]

# Új oszlop: átugrási ráta százalékban
sk_stats["Átugrási ráta (%)"] = 100 * sk_stats["skips"] / sk_stats["n"]

# Oszlopok átnevezése magyarra a megjelenítéshez
sk_stats = sk_stats.rename(columns={
    "artist": "Előadó",
    "n": "Mintaszám",
    "skips": "Átugrások",
    "total_min": "Teljes hossz (perc)"
})

# Legtöbbet átugrott előadók (top 15)
res3_high = sk_stats.sort_values("Átugrási ráta (%)", ascending=False).head(15)

# Legkevésbé átugrott előadók (top 15)
res3_low = sk_stats.sort_values("Átugrási ráta (%)", ascending=True).head(15)

# Eredmények megjelenítése kerekített értékekkel
print("Legmagasabb átugrási ráta (top 15):")
display(res3_high.round({"Átugrási ráta (%)": 2, "Teljes hossz (perc)": 1}))

print("\nLegalacsonyabb átugrási ráta (top 15):")
display(res3_low.round({"Átugrási ráta (%)": 2, "Teljes hossz (perc)": 1}))


Legmagasabb átugrási ráta (top 15):


,Előadó,Mintaszám,Átugrások,Teljes hossz (perc),Átugrási ráta (%)
77,AWS,282,214,375.5,75.89
1290,Lil Frakk,294,222,338.8,75.51
2464,gnash,167,121,212.7,72.46
1226,Krúbi,2037,1462,3917.0,71.77
230,BEATó,442,311,456.3,70.36
675,Dé:Nash,306,214,533.5,69.93
797,Frank Ocean,248,173,323.2,69.76
1535,Munn,175,118,231.3,67.43
469,Chris Brown,191,127,237.4,66.49
808,Future,176,115,197.7,65.34



Legalacsonyabb átugrási ráta (top 15):


,Előadó,Mintaszám,Átugrások,Teljes hossz (perc),Átugrási ráta (%)
1266,League of Legends,160,21,199.0,13.12
1997,Skylar Grey,272,62,640.0,22.79
336,Bo Burnham,815,209,1728.2,25.64
1398,Mackenzy Mackay,162,43,294.0,26.54
1308,Lilly Wood and The Prick,236,70,292.0,29.66
690,Ed Sheeran,522,177,689.9,33.91
1665,OneRepublic,211,73,377.6,34.60
816,GAYLE,210,73,257.6,34.76
1742,Polo G,719,251,1150.1,34.91
1277,Lewis Capaldi,164,58,336.4,35.37


In [63]:
# Lekérdezés 3) Podcast elköteleződés – műsoronként
# Csak podcast sorokat veszünk, és a podcast_show szerint aggregálunk.

pods = df[df["content_type"] == "podcast"].copy()

p4 = (
    pods.groupby("podcast_show")
        .agg(total_min=("min", "sum"),
             avg_min=("min", "mean"),
             median_min=("min", "median"),
             episodes=("podcast_episode", "nunique"),
             days=("date", "nunique"))
        .reset_index()
)

p4 = p4[p4["podcast_show"].notna() & (p4["podcast_show"] != "")]
res4 = p4.sort_values("total_min", ascending=False).head(15)
res4.round({"total_min": 1, "avg_min": 1, "median_min": 1})

,podcast_show,total_min,avg_min,median_min,episodes,days
28,Trash Taste Podcast,681.0,24.3,22.0,9,14
0,And That's Why We Drink,425.4,10.6,3.1,11,13
10,Last Podcast On The Left,412.0,11.8,3.6,12,14
24,The Misfits Podcast,407.4,13.1,5.1,8,11
18,Slightly Offensive with Elijah Schaffer,140.1,6.7,0.2,9,8
16,Science Vs,139.8,11.7,10.9,6,5
12,"My Brother, My Brother And Me",129.4,14.4,6.9,5,7
3,Chuckle Sandwich,118.4,14.8,17.6,3,4
5,GOONS,75.1,15.0,2.8,5,4
4,Dropouts,74.2,18.6,8.4,2,3


In [64]:
# 4) Offline vs. online hallgatás – havi bontás
# Cél: Havi bontásban megnézni, mennyi idő ment offline vs. online lejátszásra,
#      és ebből kiszámolni az offline arányát (%).

usage = df.copy()

# Az "offline" mező jelzi, hogy a lejátszás offline módban történt-e (True/False).
# Egyes exportokból hiányozhat vagy lehetnek benne hiányzó értékek → alapértelmezés: False.
usage["offline"] = usage.get("offline", False).fillna(False).astype(bool)

# Havi (month) és offline státusz (offline) szerinti összesítés:
# total_min = az adott hónapban offline/online összesített percek
monthly_usage = (
    usage.groupby(["month", "offline"])
         .agg(total_min=("min", "sum"))
         .reset_index()
)

# Pivot: sorok = hónapok, oszlopok = offline státusz (True/False), érték = total_min (perc)
monthly_usage_pivot = (
    monthly_usage.pivot_table(index="month", columns="offline", values="total_min", fill_value=0)
                  .sort_index()  # időrend
)

# Biztonság kedvéért gondoskodunk róla, hogy mindkét oszlop (True/False) létezzen
if True not in monthly_usage_pivot.columns:
    monthly_usage_pivot[True] = 0
if False not in monthly_usage_pivot.columns:
    monthly_usage_pivot[False] = 0

# Offline arány (%) = offline percek / (offline + online percek) * 100
monthly_usage_pivot["offline_share_%"] = (
    100 * monthly_usage_pivot[True] / (monthly_usage_pivot[True] + monthly_usage_pivot[False]).replace(0, np.nan)
)

# Kimeneti táblázat: magyar oszlopnevekkel és kerekítéssel
offline_online_by_month = (
    monthly_usage_pivot.reset_index()
                       .rename(columns={
                           "month": "Hónap",
                           True: "Offline (perc)",
                           False: "Online (perc)",
                           "offline_share_%": "Offline arány (%)"
                       })
                       .round({"Offline (perc)": 1, "Online (perc)": 1, "Offline arány (%)": 1})
)

# Megjelenítés
display(offline_online_by_month) 

# Megjeleníteni a legnagyobb offline arányú hónapokat, top 10

display(offline_online_by_month.sort_values("Offline arány (%)", ascending=False).head(10))


offline,Hónap,Online (perc),Offline (perc),Offline arány (%)
0,2018-07,38.8,0.0,0.0
1,2018-08,17.7,286.1,94.2
2,2018-09,1.1,0.0,0.0
3,2019-01,1.0,0.0,0.0
4,2019-10,220.0,0.0,0.0
...,...,...,...,...
58,2025-05,4047.0,215.8,5.1
59,2025-06,7890.1,29.1,0.4
60,2025-07,8503.4,15.8,0.2
61,2025-08,6965.0,11.2,0.2


offline,Hónap,Online (perc),Offline (perc),Offline arány (%)
1,2018-08,17.7,286.1,94.2
7,2020-02,23.1,15.8,40.6
8,2020-03,161.9,79.6,33.0
12,2021-07,1035.8,168.7,14.0
35,2023-06,3046.4,340.2,10.0
14,2021-09,1610.5,169.4,9.5
23,2022-06,2480.1,197.1,7.4
9,2020-08,477.6,38.3,7.4
6,2020-01,345.7,26.4,7.1
16,2021-11,843.7,47.7,5.4


In [65]:
# 6) Platform / Eszköz használat – havi megoszlás és trend + éves Android/Windows összesítés

# --- Havi megoszlás és trend (eredeti logika, beszédes nevekkel) ---

platform_usage_monthly = (
    df.groupby(["month", "platform_simple"])
      .agg(total_min=("min", "sum"))
      .reset_index()
)

# Havi összesített perc (offline+online mindegy, itt eszköz/platform dimenzió a lényeg)
platform_usage_monthly["month_total"] = (
    platform_usage_monthly.groupby("month")["total_min"].transform("sum")
)

# Havi részesedés a platformonkénti percekből
platform_usage_monthly["share_%"] = (
    100 * platform_usage_monthly["total_min"] / platform_usage_monthly["month_total"].replace(0, np.nan)
)

# Top platform-sorrend a legutóbbi 12 hónap összesített percei alapján
recent_months = sorted(df["month"].unique())[-12:]
recent_platform_order = (
    platform_usage_monthly[platform_usage_monthly["month"].isin(recent_months)]
    .groupby("platform_simple")["total_min"]
    .sum()
    .sort_values(ascending=False)
    .index
    .tolist()
)

# Stabil rendezés: a legutóbbi 12 hónap alapján képzett sorrend, a maradék a lista végére kerül
order_map = {p: i for i, p in enumerate(recent_platform_order)}
platform_usage_monthly_sorted = (
    platform_usage_monthly
      .assign(_order=platform_usage_monthly["platform_simple"].map(order_map).fillna(len(recent_platform_order)))
      .sort_values(["month", "_order", "platform_simple"])
      .drop(columns="_order")
)

# Kimenet: magyar oszlopnevek (csak megjelenítéshez)
platform_share_by_month = (
    platform_usage_monthly_sorted
      .rename(columns={
          "month": "Hónap",
          "platform_simple": "Platform",
          "total_min": "Összes perc",
          "month_total": "Havi összes perc",
          "share_%": "Részesedés (%)"
      })
      .round({"Összes perc": 1, "Havi összes perc": 1, "Részesedés (%)": 2})
)

# Például:
# display(platform_share_by_month.head(20))


# --- ÉVES összesítés: Android és Windows mennyisége és részesedése évenként ---

# Robusztus év-képzés a 'month' mezőből (lehet string / Period / datetime)
_year = pd.to_datetime(df["month"].astype(str), errors="coerce").dt.year

platform_usage_yearly = (
    df.assign(year=_year)
      .groupby(["year", "platform_simple"])
      .agg(total_min=("min", "sum"))
      .reset_index()
)

# Éves összes perc (összes platform)
platform_usage_yearly["year_total"] = (
    platform_usage_yearly.groupby("year")["total_min"].transform("sum")
)

# Éves részesedés platformonként
platform_usage_yearly["share_%"] = (
    100 * platform_usage_yearly["total_min"] / platform_usage_yearly["year_total"].replace(0, np.nan)
)

# Kifejezetten Android és Windows kiválogatása (kis/nagybetűktől független)
platform_usage_yearly_aw = (
    platform_usage_yearly
      .assign(platform_lower=platform_usage_yearly["platform_simple"].astype(str).str.lower())
      .loc[lambda d: d["platform_lower"].isin(["android", "windows"])]
)

# Kedvesebb platformcímkék a kimenethez
name_map = {"android": "Android", "windows": "Windows"}
platform_usage_yearly_aw["Platform"] = platform_usage_yearly_aw["platform_lower"].map(name_map)

# 1) Éves ÖSSZES PERC  (Android/Windows oszlopok)
yearly_android_windows_minutes = (
    platform_usage_yearly_aw
      .pivot_table(index="year", columns="Platform", values="total_min", aggfunc="sum", fill_value=0)
      .reset_index()
      .rename(columns={"year": "Év"})
      .round(1)
)



display(yearly_android_windows_minutes)



Platform,Év,Android,Windows
0,2018,343.7,0.0
1,2019,1.4,316.5
2,2020,763.3,439.1
3,2021,6502.0,1895.5
4,2022,20272.6,17640.8
5,2023,15575.0,34067.1
6,2024,21052.6,50897.4
7,2025,13292.0,37680.2


In [66]:
# Mindent UTC-s datetime-re konvertálunk (tz-aware), bármi is volt korábban
tracks_ts = tracks.copy()
tracks_ts["ts"] = pd.to_datetime(tracks_ts["ts"], errors="coerce", utc=True)

# 1) Előadónként az első lejátszás időpontja
first_listen_by_artist = (
    tracks_ts.dropna(subset=["artist", "ts"])
             .groupby("artist")["ts"]
             .min()
             .reset_index(name="first_ts")
)

# 2) Év
first_listen_by_artist["first_year"] = first_listen_by_artist["first_ts"].dt.year

# 3) Éves összesítés: hány új előadó volt az adott évben
new_artists_yearly = (
    first_listen_by_artist.groupby("first_year")
                          .size()
                          .reset_index(name="new_artists")
                          .sort_values("first_year")
)

# 4) Kimenet: magyar oszlopnevek
new_artists_by_year = new_artists_yearly.rename(columns={
    "first_year": "Év (első hallgatás)",
    "new_artists": "Új előadók száma"
})

display(new_artists_by_year)


,Év (első hallgatás),Új előadók száma
0,2018,3
1,2019,57
2,2021,427
3,2022,657
4,2023,616
5,2024,460
6,2025,336


In [67]:
# 9) „Evergreen” dalok – legalább 5 különböző napon hallgatva
# Cél: Dalonként (előadó + cím) megszámolni, hány KÜLÖNBÖZŐ napon szerepeltek,
#      majd a legtartósabban visszatérő ("evergreen") darabokat listázni.

# 0) Forrás szűrése: csak valós előadó és dalcím
songs_with_keys = tracks.dropna(subset=["track", "artist"]).copy()

# 1) Gondoskodjunk róla, hogy a "date" valóban nap pontosságú legyen (időpont levágása, ha lenne)
#    - ha string/datetime: to_datetime -> .dt.date
#    - ha már nap szintű, ez nem ront el semmit
songs_with_keys["_day"] = pd.to_datetime(songs_with_keys["date"], errors="coerce").dt.date

# 2) Előadó + dal szinten aggregálunk:
#    - days: hány KÜLÖNBÖZŐ napon hallgattad (nunique az "_day"-re)
#    - total_min: összes hallgatott perc
evergreen_stats = (
    songs_with_keys
      .groupby(["artist", "track"])
      .agg(
          days=("_day", "nunique"),
          total_min=("min", "sum")
      )
      .reset_index()
)

# 3) Csak azok a dalok maradjanak, amelyek legalább 5 különböző napon szerepeltek
evergreen_filtered = evergreen_stats[evergreen_stats["days"] >= 5].copy()

# 4) Rangsorolás:
#    - először a napok száma szerint csökkenően
#    - azonos napoknál az összes perc szerint csökkenően
#    - stabilitás kedvéért utána név szerint is (opcionális)
evergreen_top20 = (
    evergreen_filtered
      .sort_values(["days", "total_min", "artist", "track"], ascending=[False, False, True, True])
      .head(20)
)

# 5) Kimenet: magyar oszlopnevek + kerekítés
evergreen_top20_hu = (
    evergreen_top20
      .rename(columns={
          "artist": "Előadó",
          "track": "Dal",
          "days": "Napok száma",
          "total_min": "Összes perc"
      })
      .round({"Összes perc": 1})
)

# Megjelenítés
display(evergreen_top20_hu)


,Előadó,Dal,Napok száma,Összes perc
2353,Eminem,Lucky You (feat. Joyner Lucas),241,400.7
2439,Eminem,Till I Collapse,234,566.7
7568,XXXTENTACION,SAD!,233,241.9
5203,NF,DRIFTING,230,677.8
5232,NF,Let You Down,228,545.8
5244,NF,Nate,225,789.2
5260,NF,Remember This,221,608.7
2061,EDEN,Wake Up,210,694.9
5214,NF,Hate Myself,200,672.7
6571,Skylar Grey,"Last One Standing (feat. Polo G, Mozzy & Emine...",195,621.6


In [68]:
# 10) Hallgatási „streak” – leghosszabb megszakítás nélküli nap-sorozat
# Cél: Meghatározni, mi volt a leghosszabb sorozat, amikor minden nap hallgattál valamit.

# 1) Az összes aktív nap kigyűjtése (amikor volt hallgatás)
listening_days = pd.Series(sorted(df["date"].dropna().unique()))
listening_days = pd.to_datetime(listening_days).dt.date  # csak nap pontosság

# 2) „Trükk”: ha kivonjuk az indexet napokban, akkor a különbség konstans lesz a folytonos szakaszokban
groups = listening_days - pd.to_timedelta(range(len(listening_days)), unit="D")

# 3) Sorozatok meghatározása: kezdőnap, zárónap, hossz (napok száma)
streaks = (
    listening_days.groupby(groups)
                  .agg(start=("min"), end=("max"), length=("size"))
                  .reset_index(drop=True)
)

# 4) Magyar oszlopnevek
streaks_hu = streaks.rename(columns={
    "start": "Kezdőnap",
    "end": "Zárónap",
    "length": "Sorozat hossza (nap)"
})

# 5) Leghosszabb sorozat kiválasztása
longest_streak = streaks_hu.sort_values("Sorozat hossza (nap)", ascending=False).head(1)

display(longest_streak)


c:\Users\gondo\anaconda3\lib\site-packages\pandas\core\arrays\datetimelike.py:1189: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  warnings.warn(


,Kezdőnap,Zárónap,Sorozat hossza (nap)
76,2023-11-22,2025-02-14,451
